In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install sagemaker boto3 awscli

In [ ]:
!aws s3 ls --profile asher

In [ ]:
local_dir = "./data"

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.local import LocalSession
role = "arn:aws:iam::345594598345:role/service-role/AmazonSageMaker-ExecutionRole-20250707T163996" # get_execution_role() #
print("Role: ", role)
# local_sess = LocalSession()  # for local session
# local_sess.config = {'local': {'local_code': True}}

sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1", profile_name="asher")) #name of sso session
# #for remote session
settings = dict(
    sagemaker_session=sess,
    role=role,
    instance_type="ml.t3.large",
    dependencies='./requirements.txt',
)

bucket_name = sess.default_bucket()
print(bucket_name)

Uploaded training, testing and validation datasets to S3 bucket

In [ ]:
# !aws s3 cp ./data s3://{bucket_name}/prostate158 --recursive --profile asher
# print("Done!")

Upload weights of pre-trained UNet model

In [ ]:
# !aws s3 cp models/tumor.pt s3://{bucket_name}/models/tumor.pt --profile asher

In [ ]:
if bucket_name:
    train_channel = f"s3://{bucket_name}/prostate158/prostate158_train/train"
    test_channel = f"s3://{bucket_name}/prostate158/prostate158_test/test"
    model_channel = f"s3://{bucket_name}/models/tumor.pt"
inputs = {"train": train_channel, "test": test_channel, "model": model_channel}
print(inputs)

In [ ]:
%%time
from sagemaker.pytorch.estimator import PyTorch
import os

image = "763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:2.6.0-cpu-py312"

def run_training():
    estimator = PyTorch(
        image_uri=image,
        entry_point="train_gpt.py",
        source_dir=".",
        role=role,
        region="ap-southeast-1",
        instance_type="ml.t3.large",
        instance_count=1,
        framework_version="2.6.0",
        py_version="py312",
        enable_spot_training=True,  # for cost savings
        max_run=3600,
        input_mode="File",
        sagemaker_session=sess,
        inputs={
            "training": train_channel,
            "model": model_channel,
            "training_config": os.environ.get("SM_CHANNEL_TRAINING_CONFIG", "./tumor.yaml"),
        }
    )
    estimator.fit(inputs)

run_training()